In [15]:
import pandas as pd
import numpy as np
import biogeme as biogeme
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme import models
import biogeme.messaging as msg
#from biogeme.expressions import Beta
from biogeme.expressions import (
    Beta,
    DefineVariable,
    bioDraws,
    PanelLikelihoodTrajectory,
    MonteCarlo,
    log,
    Derive
)
import math
from datetime import datetime
import pyreadr
import pickle


In [2]:
# rerun mode choice model for calibration parameters later

micro_pool = pd.read_csv('micro_pool_socio_bio2up.csv')
micro_pool['popdensityk']= micro_pool['popdensity']/1000

database = db.Database('micro', micro_pool)
# They are organized as panel data. The variable who identifies each individual.
#database.panel("who") # remember to sort data by individual

globals().update(database.variables)

# S and RND ascs are for panel data. Ignore for calibration
ASC_CAR = Beta('ASC_CAR', 0, None, None, 1)
ASC_CAR_S = Beta('ASC_CAR_S', 1, None, None, 0)
ASC_CAR_RND = ASC_CAR + ASC_CAR_S * bioDraws('ASC_CAR_RND', 'NORMAL_ANTI')

ASC_TRANSIT = Beta('ASC_TRANSIT', 0, None, None, 0)
ASC_TRANSIT_S = Beta('ASC_TRANSIT_S', 1, None, None, 0)
ASC_TRANSIT_RND = ASC_TRANSIT + ASC_TRANSIT_S * bioDraws('ASC_TRANSIT_RND', 'NORMAL_ANTI')

ASC_RH = Beta('ASC_RH', 0, None, None, 0)
ASC_RH_S = Beta('ASC_RH_S', 1, None, None, 0)
ASC_RH_RND = ASC_RH + ASC_RH_S * bioDraws('ASC_RH_RND', 'NORMAL_ANTI')

ASC_WALK = Beta('ASC_WALK', 0, None, None, 0)
ASC_WALK_S = Beta('ASC_WALK_S', 1, None, None, 0)
ASC_WALK_RND = ASC_WALK + ASC_WALK_S * bioDraws('ASC_WALK_RND', 'NORMAL_ANTI')

ASC_BIKE = Beta('ASC_BIKE', 0, None, None, 0)
ASC_BIKE_S = Beta('ASC_BIKET_S', 1, None, None, 0)
ASC_BIKE_RND = ASC_BIKE + ASC_BIKE_S * bioDraws('ASC_TRANSIT_RND', 'NORMAL_ANTI')

ASC_SCOOTER = Beta('ASC_SCOOTER', 0, None, None, 0)
ASC_SCOOTER_S = Beta('ASC_SCOOTER_S', 1, None, None, 0)
ASC_SCOOTER_RND = ASC_SCOOTER + ASC_SCOOTER_S * bioDraws('ASC_SCOOTER_RND', 'NORMAL_ANTI')

ASC_DLBIKE = Beta('ASC_DLBIKE', 0, None, None, 0)
ASC_DLBIKE_S = Beta('ASC_DLBIKE_S', 1, None, None, 0)
ASC_DLBIKE_RND = ASC_DLBIKE + ASC_DLBIKE_S * bioDraws('ASC_DLBIKE_RND', 'NORMAL_ANTI')

ASC_DKBIKE = Beta('ASC_DKBIKE', 0, None, None, 0)
ASC_DKBIKE_S = Beta('ASC_DKBIKE_S', 1, None, None, 0)
ASC_DKBIKE_RND = ASC_DKBIKE + ASC_DKBIKE_S * bioDraws('ASC_DKBIKE_RND', 'NORMAL_ANTI')

ASC_SCTRANSIT = Beta('ASC_SCTRANSIT', 0, None, None, 0)
ASC_SCTRANSIT_S = Beta('ASC_SCTRANSIT_S', 1, None, None, 0)
ASC_SCTRANSIT_RND = ASC_SCTRANSIT + ASC_SCTRANSIT_S * bioDraws('ASC_SCTRANSIT_RND', 'NORMAL_ANTI')


B_carTIME = Beta('B_carTIME', 0, None, None, 0) #name, default value, lower bound, upper bound, estimate(Y0,N1)
B_transitTIME = Beta('B_transitTIME', 0, None, None, 0)
B_rhTIME = Beta('B_rhTIME', 0, None, None, 0)
B_walkTIME = Beta('B_walkTIME', 0, None, None, 0)
B_bikeTIME = Beta('B_bikeTIME', 0, None, None, 0)

B_scooterTIME = Beta('B_scooterTIME', 0, None, None, 0)
B_sctransitTIME = Beta('B_sctransitTIME', 0, None, None, 0)

B_access = Beta('B_access', 0, None, None, 0)
B_drop =Beta('B_drop', 0, None, None, 0)
B_waitav = Beta('B_waitav', 0, None, None, 0)
B_AV = Beta('B_AV', 0, None, None, 0)

B_PRCP1 = Beta('B_PRCP1', 0, None, None, 0)#Heavy rain
B_PRCP2 = Beta('B_PRCP2', 0, None, None, 0)#Light rain, ref: No rain

B_bikePRCP1 = Beta('B_bikePRCP1', 0, None, None, 0)#Heavy rain
B_bikePRCP2 = Beta('B_bikePRCP2', 0, None, None, 0)#Light rain, ref: No rain

B_scooterPRCP1 = Beta('B_scooterPRCP1', 0, None, None, 0)#Heavy rain
B_scooterPRCP2 = Beta('B_scooterPRCP2', 0, None, None, 0)#Light rain, ref: No rain


B_bikeBKLN2 = Beta('B_bikeBKLN2', 0, None, None, 0)#80% bikelane
B_bikeBKLN3 = Beta('B_bikeBKLN3', 0, None, None, 0)#half bikelane
B_bikeBKLN4 = Beta('B_bikeBKLN4', 0, None, None, 0)#no bikelane, ref: full bikelane

B_scooterBKLN2 = Beta('B_scooterBKLN2', 0, None, None, 0)#80% bikelane
B_scooterBKLN3 = Beta('B_scooterBKLN3', 0, None, None, 0)#half bikelane
B_scooterBKLN4 = Beta('B_scooterBKLN4', 0, None, None, 0)#no bikelane, ref: full bikelane

B_bikeBKLNn = Beta('B_bikeBKLNn', 0, None, None, 0)
B_scooterBKLNn = Beta('B_scooterBKLNn', 0, None, None, 0)

B_TEMPbike = Beta('B_TEMPbike', 0, None, None, 0)
B_TEMPsqbike = Beta('B_TEMPsqbike', 0, None, None, 0)

B_TEMPsc = Beta('B_TEMPsc', 0, None, None, 0)
B_TEMPsqsc = Beta('B_TEMPsqsc', 0, None, None, 0)

B_carHBW = Beta('B_carHBW', 0, None, None, 0)
B_carHBSHOP = Beta('B_carHBSHOP', 0, None, None, 0)
B_carHBSOC = Beta('B_carHBSOC', 0, None, None, 0)
B_carHBO = Beta('B_carHBO', 0, None, None, 0)
B_carNHB = Beta('B_carNHB', 0, None, None, 0)


B_carPURPHome = Beta('B_carPURPHome', 0, None, None, 0)#go back home
B_carPURPShop = Beta('B_carPURPShop', 0, None, None, 0)#shopping
B_carPURPErra = Beta('B_carPURPErra', 0, None, None, 0)#errand
B_carPURPScho = Beta('B_carPURPScho', 0, None, None, 0)#school/church
B_carPURPSoci = Beta('B_carPURPSoci', 0, None, None, 0)#social

B_carorigHome = Beta('B_carorigHome', 0, None, None, 0)
B_carorigShop = Beta('B_carorigShop', 0, None, None, 0)
B_carorigErra = Beta('B_carorigErra', 0, None, None, 0)
B_carorigScho = Beta('B_carorigScho', 0, None, None, 0)
B_carorigSoci = Beta('B_carorigSoci', 0, None, None, 0)

B_transitHBW = Beta('B_transitHBW', 0, None, None, 0)
B_transitHBSHOP = Beta('B_transitHBSHOP', 0, None, None, 0)
B_transitHBSOC = Beta('B_transitHBSOC', 0, None, None, 0)
B_transitHBO = Beta('B_transitHBO', 0, None, None, 0)
B_transitNHB = Beta('B_transitNHB', 0, None, None, 0)

B_transitPURPHome = Beta('B_transitPURPHome', 0, None, None, 0)#go back home
B_transitPURPShop = Beta('B_transitPURPShop', 0, None, None, 0)#shopping
B_transitPURPErra = Beta('B_transitPURPErra', 0, None, None, 0)#errand
B_transitPURPScho = Beta('B_transitPURPScho', 0, None, None, 0)#school/church
B_transitPURPSoci = Beta('B_transitPURPSoci', 0, None, None, 0)#social

B_transitorigHome = Beta('B_transitorigHome', 0, None, None, 0)
B_transitorigShop = Beta('B_transitorigShop', 0, None, None, 0)
B_transitorigErra = Beta('B_transitorigErra', 0, None, None, 0)
B_transitorigScho = Beta('B_transitorigScho', 0, None, None, 0)
B_transitorigSoci = Beta('B_transitorigSoci', 0, None, None, 0)

B_rhHBW = Beta('B_rhHBW', 0, None, None, 0)
B_rhHBSHOP = Beta('B_rhHBSHOP', 0, None, None, 0)
B_rhHBSOC = Beta('B_rhHBSOC', 0, None, None, 0)
B_rhHBO = Beta('B_rhHBO', 0, None, None, 0)
B_rhNHB = Beta('B_rhNHB', 0, None, None, 0)

B_rhPURPHome = Beta('B_rhPURPHome', 0, None, None, 0)#go back home
B_rhPURPShop = Beta('B_rhPURPShop', 0, None, None, 0)#shopping
B_rhPURPErra = Beta('B_rhPURPErra', 0, None, None, 0)#errand
B_rhPURPScho = Beta('B_rhPURPScho', 0, None, None, 0)#school/church
B_rhPURPSoci = Beta('B_rhPURPSoci', 0, None, None, 0)#social

B_rhorigHome = Beta('B_rhorigHome', 0, None, None, 0)
B_rhorigShop = Beta('B_rhorigShop', 0, None, None, 0)
B_rhorigErra = Beta('B_rhorigErra', 0, None, None, 0)
B_rhorigScho = Beta('B_rhorigScho', 0, None, None, 0)
B_rhorigSoci = Beta('B_rhorigSoci', 0, None, None, 0)

B_walkHBW = Beta('B_walkHBW', 0, None, None, 0)
B_walkHBSHOP = Beta('B_walkHBSHOP', 0, None, None, 0)
B_walkHBSOC = Beta('B_walkHBSOC', 0, None, None, 0)
B_walkHBO = Beta('B_walkHBO', 0, None, None, 0)
B_walkNHB = Beta('B_walkNHB', 0, None, None, 0)

B_walkPURPHome = Beta('B_walkPURPHome', 0, None, None, 0)#go back home
B_walkPURPShop = Beta('B_walkPURPShop', 0, None, None, 0)#shopping
B_walkPURPErra = Beta('B_walkPURPErra', 0, None, None, 0)#errand
B_walkPURPScho = Beta('B_walkPURPScho', 0, None, None, 0)#school/church
B_walkPURPSoci = Beta('B_walkPURPSoci', 0, None, None, 0)#social

B_walkorigHome = Beta('B_walkorigHome', 0, None, None, 0)
B_walkorigShop = Beta('B_walkorigShop', 0, None, None, 0)
B_walkorigErra = Beta('B_walkorigErra', 0, None, None, 0)
B_walkorigScho = Beta('B_walkorigScho', 0, None, None, 0)
B_walkorigSoci = Beta('B_walkorigSoci', 0, None, None, 0)

B_bikeHBW = Beta('B_bikeHBW', 0, None, None, 0)
B_bikeHBSHOP = Beta('B_bikeHBSHOP', 0, None, None, 0)
B_bikeHBSOC = Beta('B_bikeHBSOC', 0, None, None, 0)
B_bikeHBO = Beta('B_bikeHBO', 0, None, None, 0)
B_bikeNHB = Beta('B_bikeNHB', 0, None, None, 0)

B_bikePURPHome = Beta('B_bikePURPHome', 0, None, None, 0)#go back home
B_bikePURPShop = Beta('B_bikePURPShop', 0, None, None, 0)#shopping
B_bikePURPErra = Beta('B_bikePURPErra', 0, None, None, 0)#errand
B_bikePURPScho = Beta('B_bikePURPScho', 0, None, None, 0)#school/church
B_bikePURPSoci = Beta('B_bikePURPSoci', 0, None, None, 0)#social

B_bikeorigHome = Beta('B_bikeorigHome', 0, None, None, 0)
B_bikeorigShop = Beta('B_bikeorigShop', 0, None, None, 0)
B_bikeorigErra = Beta('B_bikeorigErra', 0, None, None, 0)
B_bikeorigScho = Beta('B_bikeorigScho', 0, None, None, 0)
B_bikeorigSoci = Beta('B_bikeorigSoci', 0, None, None, 0)

B_microHBW = Beta('B_microHBW', 0, None, None, 0)
B_microHBSHOP = Beta('B_microHBSHOP', 0, None, None, 0)
B_microHBSOC = Beta('B_microHBSOC', 0, None, None, 0)
B_microHBO = Beta('B_microHBO', 0, None, None, 0)
B_microNHB = Beta('B_microNHB', 0, None, None, 0)

B_microPURPHome = Beta('B_microPURPHome', 0, None, None, 0)#go back home
B_microPURPShop = Beta('B_microPURPShop', 0, None, None, 0)#shopping
B_microPURPErra = Beta('B_microPURPErra', 0, None, None, 0)#errand
B_microPURPScho = Beta('B_microPURPScho', 0, None, None, 0)#school/church
B_microPURPSoci = Beta('B_microPURPSoci', 0, None, None, 0)#social

B_microorigHome = Beta('B_microorigHome', 0, None, None, 0)
B_microorigShop = Beta('B_microorigShop', 0, None, None, 0)
B_microorigErra = Beta('B_microorigErra', 0, None, None, 0)
B_microorigScho = Beta('B_microorigScho', 0, None, None, 0)
B_microorigSoci = Beta('B_microorigSoci', 0, None, None, 0)


B_carage = Beta('B_carage', 0, None, None, 0)
B_transitage = Beta('B_transitage', 0, None, None, 0)
B_rhage = Beta('B_rhage', 0, None, None, 0)
B_walkage = Beta('B_walkage', 0, None, None, 0)
B_bikeage = Beta('B_bikeage', 0, None, None, 0)
B_microage = Beta('B_microage', 0, None, None, 0)

B_caragesq = Beta('B_caragesq', 0, None, None, 0)
B_transitagesq = Beta('B_transitagesq', 0, None, None, 0)
B_rhagesq = Beta('B_rhagesq', 0, None, None, 0)
B_walkagesq = Beta('B_walkagesq', 0, None, None, 0)
B_bikeagesq = Beta('B_bikeagesq', 0, None, None, 0)
B_microagesq = Beta('B_microagesq', 0, None, None, 0)

B_carFe = Beta('B_carFe', 0, None, None, 0)
B_transitFe = Beta('B_transitFe', 0, None, None, 0)
B_rhFe = Beta('B_rhFe', 0, None, None, 0)
B_walkFe = Beta('B_walkFe', 0, None, None, 0)
B_bikeFe = Beta('B_bikeFe', 0, None, None, 0)
B_scooterFe = Beta('B_scooterFe', 0, None, None, 0)
B_sharedbikeFe = Beta('B_sharedbikeFe', 0, None, None, 0)


B_carincomek = Beta('B_carincomek', 0, None, None, 0)
B_transitincomek = Beta('B_transitincomek', 0, None, None, 0)
B_rhincomek = Beta('B_rhincomek', 0, None, None, 0)
B_walkincomek = Beta('B_walkincomek', 0, None, None, 0)
B_bikeincomek = Beta('B_bikeincomek', 0, None, None, 0)
B_microincomek = Beta('B_microincomek', 0, None, None, 0)

B_carhedu = Beta('B_carhedu', 0, None, None, 0)
B_transithedu = Beta('B_transithedu', 0, None, None, 0)
B_rhhedu = Beta('B_rhhedu', 0, None, None, 0)
B_walkhedu = Beta('B_walkhedu', 0, None, None, 0)
B_bikehedu = Beta('B_bikehedu', 0, None, None, 0)
B_microhedu = Beta('B_microhedu', 0, None, None, 0)

B_caremp = Beta('B_caremp', 0, None, None, 0)
B_transitemp = Beta('B_transitemp', 0, None, None, 0)
B_rhemp = Beta('B_rhemp', 0, None, None, 0)
B_walkemp = Beta('B_walkemp', 0, None, None, 0)
B_bikeemp = Beta('B_bikeemp', 0, None, None, 0)
B_microemp = Beta('B_microemp', 0, None, None, 0)


B_carhhsize = Beta('B_carhhsize', 0, None, None, 0)
B_transithhsize = Beta('B_transithhsize', 0, None, None, 0)
B_rhhhsize = Beta('B_rhhhsize', 0, None, None, 0)
B_walkhhsize = Beta('B_walkhhsize', 0, None, None, 0)
B_bikehhsize = Beta('B_bikehhsize', 0, None, None, 0)
B_microhhsize = Beta('B_microhhsize', 0, None, None, 0)


B_carPBIKE = Beta('B_carPBIKE', 0, None, None, 0)
B_transitPBIKE = Beta('B_transitPBIKE', 0, None, None, 0)
B_rhPBIKE = Beta('B_rhPBIKE', 0, None, None, 0)
B_walkPBIKE = Beta('B_walkPBIKE', 0, None, None, 0)
B_bikePBIKE = Beta('B_bikePBIKE', 0, None, None, 0)
B_microPBIKE = Beta('B_microPBIKE', 0, None, None, 0)

B_carpop = Beta('B_carpop', 0, None, None, 0)
B_transitpop = Beta('B_transitpop', 0, None, None, 0)
B_rhpop = Beta('B_rhpop', 0, None, None, 0)
B_walkpop = Beta('B_walkpop', 0, None, None, 0)
B_bikepop = Beta('B_bikepop', 0, None, None, 0)
B_micropop = Beta('B_micropop', 0, None, None, 0)


B_cost = Beta('B_cost', 0, None, None, 0)
B_costadj = Beta('B_costadj', 0, None, None, 0)

# Definition of the utility functions
V1 = (ASC_CAR#_RND 
      + B_carTIME * cartime 
      + B_carhhsize*hhsize
     )
V2 = (ASC_TRANSIT#_RND 
      + B_transitTIME * transittime 
      + B_transitHBSHOP*trippurp_HBSHOP
      + B_transitHBSOC*trippurp_HBSOC
      + B_transitHBO*trippurp_HBO
      + B_transitNHB*trippurp_NHB
      )

V3 = (ASC_RH#_RND 
      + B_rhTIME * rdtime 
      + B_rhHBSHOP*trippurp_HBSHOP
      + B_rhHBSOC*trippurp_HBSOC
      + B_rhHBO*trippurp_HBO
      + B_rhNHB*trippurp_NHB
      + B_rhemp*employ
      + B_rhhhsize*hhsize
     )

V4 = (ASC_WALK#_RND 
      + B_walkTIME * walktime 
      + B_walkHBSHOP*trippurp_HBSHOP
      + B_walkHBSOC*trippurp_HBSOC
      + B_walkHBO*trippurp_HBO
      + B_walkNHB*trippurp_NHB
      )

V5 = (ASC_BIKE#_RND 
      + B_bikeTIME * biketime
      + B_bikeHBSHOP*trippurp_HBSHOP
      + B_bikeHBSOC*trippurp_HBSOC
      + B_bikeHBO*trippurp_HBO
      + B_bikeNHB*trippurp_NHB
      + B_PRCP1*PRCP_1 + B_PRCP2*PRCP_2
      + B_bikeBKLN3*(BKLN_3 + BKLN_4)
     )

#V6 micromobility
V7 = (ASC_SCOOTER#_RND 
      + B_scooterTIME*sctime 
      + B_costadj*sccost_adj
      + B_access*SCAW*(1-AVtech) + B_waitav*SCAV*AVtech + B_AV*AVtech 
      + B_PRCP1*PRCP_1 + B_PRCP2*PRCP_2
      +B_bikeBKLN3*(BKLN_3 + BKLN_4)
      + B_microHBSHOP*trippurp_HBSHOP
      + B_microHBSOC*trippurp_HBSOC
      + B_microHBO*trippurp_HBO
      + B_microNHB*trippurp_NHB
      + B_microemp*employ
      + B_microPBIKE*bike
      + B_micropop*popdensityk
      )

V8 = (ASC_DLBIKE#_RND 
      + B_bikeTIME*dltime 
      + B_costadj*dlcost_adj 
      + B_access*DLAW
      + B_PRCP1*PRCP_1 + B_PRCP2*PRCP_2
      + B_bikeBKLN3*(BKLN_3 + BKLN_4)
      + B_microHBSHOP*trippurp_HBSHOP
      + B_microHBSOC*trippurp_HBSOC
      + B_microHBO*trippurp_HBO
      + B_microNHB*trippurp_NHB
      + B_microemp*employ
      + B_microPBIKE*bike
      + B_micropop*popdensityk
    
     )

V9 = (ASC_DKBIKE#_RND 
      + B_bikeTIME*dbtime
      + B_costadj*dbcost_adj 
      + B_access*DBAW + B_drop*DBDW 
      + B_PRCP1*PRCP_1 + B_PRCP2*PRCP_2
      + B_bikeBKLN3*(BKLN_3 + BKLN_4)
      + B_microHBSHOP*trippurp_HBSHOP
      + B_microHBSOC*trippurp_HBSOC
      + B_microHBO*trippurp_HBO
      + B_microNHB*trippurp_NHB
      + B_microemp*employ
      + B_microPBIKE*bike
      + B_micropop*popdensityk

     )

V10 = (ASC_SCTRANSIT#_RND 
       + B_scooterTIME*sttime #
       + B_transitTIME*stransittime 
       + B_costadj*sttocost_adj 
       + B_access*STAW*(1-AVtech ) + B_waitav*STAV*AVtech + B_AV*AVtech  
       + B_PRCP1*PRCP_1 + B_PRCP2*PRCP_2
       +B_bikeBKLN3*(BKLN_3 + BKLN_4)
       + B_microHBSHOP*trippurp_HBSHOP
       + B_microHBSOC*trippurp_HBSOC
       + B_microHBO*trippurp_HBO
       + B_microNHB*trippurp_NHB
       + B_microemp*employ
       + B_microPBIKE*bike
       + B_micropop*popdensityk
       
      )

V = {1: V1, 2: V2, 3:V3, 4: V4, 5: V5, 7: V7, 8: V8, 9: V9, 10: V10}

# Associate the availability conditions with the alternatives
av = {1: car_av, 2: transit_av, 3:rd_av, 4:walk_av, 5:bike_av, 
      7:scooter_av, 8:dlbike_av, 9:dkbike_av, 10:sctransit_av}

# Definition of the model. This is the contribution of each
# observation to the log likelihood function.
logprob = models.loglogit(V, av, choice)

# Create the Biogeme object
biogeme = bio.BIOGEME(database, logprob)
biogeme.modelName = '01microdata_logit_costadj_incomek_den_hbpurp_socio_bklnn_nocar'
#biogeme.modelName = '01microdata_logit_cost_incomek'
# Calculate the null log likelihood for reporting.
biogeme.calculateNullLoglikelihood(av)

# Estimate the parameters
model_results = biogeme.estimate()

# Get the results in a pandas table
pandasResults = model_results.getEstimatedParameters()
#print(pandasResults)

In [3]:
path = "/Users/tianqizou/Documents/DOE/SCOOT/ApplyModel/SmallSample/scootApply/appConstruct"
ausmsa_trip_addvar = pyreadr.read_r(path+'/'+'addVar_tracts_12420.rds')
ausmsa_trip_addvar  = ausmsa_trip_addvar[None]
ausmsa_trip_addvar.head()
ausmsa_trip_addvar.shape

(241370, 83)

In [4]:
# mimo vehicle availability
station_walktime = pd.read_csv('/Users/tianqizou/Documents/DOE/model literature/hybrid choice/aws/station_walktime.csv')
free_walktime = pd.read_csv('/Users/tianqizou/Documents/DOE/model literature/hybrid choice/aws/aus_free_walktime.csv')

aus_total = pd.read_csv('/Users/tianqizou/Documents/DOE/model literature/hybrid choice/aws/aus_march_updated.csv')


In [5]:
# check generated tracts
ausmsa_trip_addvar['TRACT'] = pd.to_numeric(ausmsa_trip_addvar['TRACT'])
ausmsa_trip_addvar['orig_tract'] = pd.to_numeric(ausmsa_trip_addvar['orig_tract'])

# this is checking for home tract
#aus_trip = ausmsa_trip_addvar[ausmsa_trip_addvar['TRACT'].isin(aus_total['GEOID10'])]
#len(pd.unique(aus_trip['TRACT']))

# this is checking for trip origin tract
aus_trip = ausmsa_trip_addvar[ausmsa_trip_addvar['orig_tract'].isin(aus_total['GEOID10'])]
len(pd.unique(aus_trip['orig_tract']))

151

In [7]:
aus_total_ingen = aus_total[aus_total['GEOID10'].isin(ausmsa_trip_addvar['orig_tract'])]
len(pd.unique(aus_total_ingen['GEOID10']))
aus_total_sum = aus_total_ingen['ID_count'].sum()
aus_total_scooter = aus_total_ingen['fid_count'].sum()
aus_total_bike = aus_total_sum-aus_total_scooter
aus_total_scooter
aus_total_bike

5304.0

In [10]:
# merge bike availability
aus_station = station_walktime[station_walktime['geoid'].isin(aus_trip['orig_tract'])]
aus_station = aus_station.rename(columns={'avg_walk_time':'docked_wt'})
aus_free = free_walktime[free_walktime['geoid'].isin(aus_trip['orig_tract'])]
aus_free_wide = aus_free.pivot(index='geoid', columns='mode', values='avg_walk_time')
aus_free_wide['geoid'] = aus_free_wide.index
aus_free_wide = aus_free_wide.rename(columns={'bike':'dockless_wt', 'scooter': 'scooter_wt'})

aus_free_wide.reset_index(drop=True, inplace=True)
print(aus_station.head())
aus_free_wide.head()
aus_free_wide.describe()

#aus_veh_avail = pd.DataFrame(aus_bike.groupby('geoid',as_index=False)['avg_walk_time'].mean())
aus_trip_veh_avail = aus_trip.merge(aus_station, left_on = 'orig_tract',right_on = 'geoid', how = 'left')
aus_trip_veh_avail = aus_trip_veh_avail.merge(aus_free_wide, left_on = 'orig_tract',right_on = 'geoid', how = 'left')
#aus_station.columns
#aus_trip_veh_avail.info()

      Unnamed: 0        geoid  docked_wt
1438        1438  48453000401   4.140898
1439        1439  48453000601   1.915190
1440        1440  48453000603   9.649647
1441        1441  48453000604   1.620575
1442        1442  48453000700   2.488615


In [11]:
# set access time for variables in the choice model
aus_trip_veh_avail['SCAW'] = aus_trip_veh_avail['scooter_wt']
aus_trip_veh_avail['DLAW'] = aus_trip_veh_avail['dockless_wt']
aus_trip_veh_avail['DBAW'] = aus_trip_veh_avail['docked_wt']
aus_trip_veh_avail['STAW'] = aus_trip_veh_avail['scooter_wt']
aus_trip_veh_avail['DBDW'] = aus_trip_veh_avail['docked_wt']

aus_trip_veh_avail.loc[aus_trip_veh_avail['scooter_wt'].isna(),['sc_av','st_av']] = 0
aus_trip_veh_avail.loc[aus_trip_veh_avail['dockless_wt'].isna(),'dl_av'] = 0
aus_trip_veh_avail.loc[aus_trip_veh_avail['docked_wt'].isna(),'db_av'] = 0

aus_trip_veh_avail.loc[aus_trip_veh_avail['scooter_wt'].isna(),['SCAW','STAW']] = 999
aus_trip_veh_avail.loc[aus_trip_veh_avail['dockless_wt'].isna(),'DLAW'] = 999
aus_trip_veh_avail.loc[aus_trip_veh_avail['docked_wt'].isna(),['DBDW','DBAW']] = 999

# set unavailability for walking time larger than 10 min
aus_trip_veh_avail.loc[aus_trip_veh_avail['scooter_wt']>10,['sc_av','st_av']] = 0
aus_trip_veh_avail.loc[aus_trip_veh_avail['dockless_wt']>10,'dl_av'] = 0
aus_trip_veh_avail.loc[aus_trip_veh_avail['docked_wt']>10,'db_av'] = 0


In [12]:
#precipitation
cbsa_prep = pd.read_csv('/Users/tianqizou/Documents/DOE/precipitation/cbsa_prep.csv')
auscode = cbsa_prep['CBSAFP']==12420
ausmo = cbsa_prep['month']==6
aus_prep = cbsa_prep.loc[auscode & ausmo, ['PRCP1','PRCP2','PRCP3']]
aus_prep


,PRCP1,PRCP2,PRCP3
653,1.692857,3.907143,24.4


In [16]:
# apply model for calibration

prob1 = models.logit(V, av, 1)
prob2 = models.logit(V, av, 2)
prob3 = models.logit(V, av, 3)
prob4 = models.logit(V, av, 4)
prob5 = models.logit(V, av, 5)
prob7 = models.logit(V, av, 7)
prob8 = models.logit(V, av, 8)
prob9 = models.logit(V, av, 9)
prob10 = models.logit(V, av, 10)
# Elasticities can be computed. We illustrate below two
# formulas. Check in the output file that they produce the same
# result.

# First, the general definition of elasticities. This illustrates the
# use of the Derive expression, and can be used with any model,
# however complicated it is. Note the quotes in the Derive opertor.
#genelas1 = Derive(prob1, 'cartime') * cartime / prob1 
#genelas2 = Derive(prob2, 'transittime') * transittime / prob2
#genelas3 = Derive(prob3, 'rdtime') * rdtime / prob3

# Second, the elasticity of logit models. See Ben-Akiva and Lerman for
# the formula

logitelas1 = car_av * (1.0 - prob1)* B_carTIME * cartime 
logitelas2 = transit_av * (1.0 - prob2)*B_transitTIME * transittime 
logitelas3 = rd_av* (1.0 - prob3)* B_rhTIME * rdtime
simulate = {
    'Prob. car': prob1,
    'Prob. transit': prob2,
    'Prob. ridehailing': prob3,
    'Prob. walk': prob4,
    'Prob. bike': prob5,
    'Prob. scooter': prob7,
    'Prob. docklessbike': prob8,
    'Prob. dockedbike': prob9,
    'Prob. scooter+transit': prob10,
    'car time logit elas.': logitelas1,
    'transit time logit elas.': logitelas2,
    'taxi time logit elas.': logitelas3,
    'car utility' : V[1]
}

### validation iteratively update ASC
## calibrate micromobility mode
# get mome mode ridership split
# aus_total_scooter = sum(aus_total['fid_count'])
# aus_total_bike = aus_total_sum-aus_total_bike


# heavy rain  #1.7days

austin_hr = aus_trip_veh_avail.copy()
austin_hr.loc[:,'PRCP_1']=1
austin_hr.loc[:,'PRCP_2']=0
austin_hr.loc[:,'PRCP_3']=0

# light rain  #3.9days
austin_lr = aus_trip_veh_avail.copy()
austin_lr.loc[:,'PRCP_1']=0
austin_lr.loc[:,'PRCP_2']=1
austin_lr.loc[:,'PRCP_3']=0

# no rain #24.4 days
austin_nr = aus_trip_veh_avail.copy()
austin_nr.loc[:,'PRCP_1']=0
austin_nr.loc[:,'PRCP_2']=0
austin_nr.loc[:,'PRCP_3']=1


austin_hr = austin_hr.drop(['user', 'age', 'gender', 'race', 'hispanic', 'usborn', 'edu',
       'student', 'work', 'zipcode','disable','trippurp','Unnamed: 0','geoid_x', 'docked_wt', 
                                              'dockless_wt', 'scooter_wt', 'geoid_y'],axis=1)
austin_lr = austin_lr.drop(['user', 'age', 'gender', 'race', 'hispanic', 'usborn', 'edu',
       'student', 'work', 'zipcode','disable','trippurp','Unnamed: 0','geoid_x', 'docked_wt', 
                                              'dockless_wt', 'scooter_wt', 'geoid_y'],axis=1)
austin_nr = austin_nr.drop(['user', 'age', 'gender', 'race', 'hispanic', 'usborn', 'edu',
       'student', 'work', 'zipcode','disable','trippurp','Unnamed: 0','geoid_x', 'docked_wt', 
                                              'dockless_wt', 'scooter_wt', 'geoid_y'],axis=1)


austin_hr = austin_hr.rename(columns={"rh_av": "rd_av", "sc_av": "scooter_av","dl_av":"dlbike_av","db_av":"dkbike_av",
                       "st_av":"sctransit_av"})

austin_lr = austin_lr.rename(columns={"rh_av": "rd_av", "sc_av": "scooter_av","dl_av":"dlbike_av","db_av":"dkbike_av",
                       "st_av":"sctransit_av"})

austin_nr = austin_nr.rename(columns={"rh_av": "rd_av", "sc_av": "scooter_av","dl_av":"dlbike_av","db_av":"dkbike_av",
                       "st_av":"sctransit_av"})

austin_hr = austin_hr.dropna()
austin_lr = austin_lr.dropna()
austin_nr = austin_nr.dropna()

austin_hr = austin_hr.astype(int)
austin_lr = austin_lr.astype(int)
austin_nr = austin_nr.astype(int)

# set micromobility availability as 
# done previously
# aus_trip_va.loc[aus_trip_va['avg_walk_time'].isna(),['sc_av','dl_av','db_av','st_av']] = 0

database_austin_hr = db.Database('ahr', austin_hr)
database_austin_lr = db.Database('alr', austin_lr)
database_austin_nr = db.Database('anr', austin_nr)
#globals().update(database_chicago.variables)

#beta_res = model_results.getBetaValues()
beta_res = pickle.load(open('beta_res_1226.pkl', 'rb'))
#beta_res = beta_res_shrk
#beta_res = beta_res_shrk
eps = 0.01
# while loop here
#i = 0
#car_els_ls = []
#transit_els_ls = []
#rh_els_ls = []

all_modes_total = len(austin_hr)*30
aus_total_bike_ms = aus_total_bike/all_modes_total
aus_total_scooter_ms = aus_total_scooter/all_modes_total

biogeme_hr = bio.BIOGEME(database_austin_hr, simulate)
biogeme_lr = bio.BIOGEME(database_austin_lr, simulate)
biogeme_nr = bio.BIOGEME(database_austin_nr, simulate)
    
results_cali_hr = biogeme_hr.simulate(theBetaValues=beta_res)
results_cali_lr = biogeme_lr.simulate(theBetaValues=beta_res)
results_cali_nr = biogeme_nr.simulate(theBetaValues=beta_res)
    
res_mean_hr = results_cali_hr.mean(axis=0)
res_mean_lr = results_cali_lr.mean(axis=0)
res_mean_nr = results_cali_nr.mean(axis=0)
    
scooter_total_hr = (res_mean_hr[5]+res_mean_hr[8])*len(austin_hr)*aus_prep['PRCP1']
scooter_total_lr = (res_mean_lr[5]+res_mean_lr[8])*len(austin_hr)*aus_prep['PRCP2']
scooter_total_nr = (res_mean_nr[5]+res_mean_nr[8])*len(austin_hr)*aus_prep['PRCP3']
    
bike_total_hr = (res_mean_hr[6]+res_mean_hr[7])*len(austin_hr)*aus_prep['PRCP1']
bike_total_lr = (res_mean_lr[6]+res_mean_lr[7])*len(austin_hr)*aus_prep['PRCP2']
bike_total_nr = (res_mean_nr[6]+res_mean_nr[7])*len(austin_hr)*aus_prep['PRCP3']


In [17]:
# total ridership in 30 days
avg_modeshare = (res_mean_hr*aus_prep['PRCP1'].values[0] + res_mean_lr*aus_prep['PRCP2'].values[0] +res_mean_nr*aus_prep['PRCP3'].values[0])/30
avg_tot_mimo_share = sum(avg_modeshare[5:9])
avg_tot_bike_share = sum(avg_modeshare[6:8])
avg_tot_scooter_share = sum(avg_modeshare[[5,8]])
print(avg_tot_bike_share)
print(avg_tot_scooter_share)

scooter_mimoshare = avg_modeshare[5]/avg_tot_scooter_share
dlbike_mimoshare = avg_modeshare[6]/avg_tot_bike_share
dkbike_mimoshare = avg_modeshare[7]/avg_tot_bike_share
sc_transit_mimoshare = avg_modeshare[8]/avg_tot_scooter_share

print(scooter_mimoshare)
print(dlbike_mimoshare)
print(dkbike_mimoshare)
print(sc_transit_mimoshare)

0.04743329915274458
0.06786531414403751
0.5098789396730838
0.7698314909136741
0.23016850908632597
0.49012106032691605


In [19]:
all_modes_total = len(austin_hr)*30/0.03
aus_total_bike_ms = aus_total_bike/all_modes_total
aus_total_scooter_ms = aus_total_scooter/all_modes_total

aus_dlbike_ms = aus_total_bike_ms*dlbike_mimoshare
aus_dkbike_ms = aus_total_bike_ms*dkbike_mimoshare
aus_scooter_ms = aus_total_scooter_ms*scooter_mimoshare
aus_sc_transit_ms = aus_total_scooter_ms*sc_transit_mimoshare

print(aus_dlbike_ms)
print(aus_dkbike_ms)
print(aus_scooter_ms)
print(aus_sc_transit_ms)


3.9916965430396585e-05
1.1934596764105433e-05
0.0021441114227726153
0.002061026808956161


In [22]:
eps = 0.001
# while loop here
i = 0
while 1:
    # apply tour to mode choice model to get probability mean for each mode
    biogeme_hr = bio.BIOGEME(database_austin_hr, simulate)
    biogeme_lr = bio.BIOGEME(database_austin_lr, simulate)
    biogeme_nr = bio.BIOGEME(database_austin_nr, simulate)
    
    results_cali_hr = biogeme_hr.simulate(theBetaValues=beta_res)
    results_cali_lr = biogeme_lr.simulate(theBetaValues=beta_res)
    results_cali_nr = biogeme_nr.simulate(theBetaValues=beta_res)
    
    res_mean_hr = results_cali_hr.mean(axis=0)
    res_mean_lr = results_cali_lr.mean(axis=0)
    res_mean_nr = results_cali_nr.mean(axis=0)
    
    scooter_total_hr = (res_mean_hr[5]+res_mean_hr[8])*len(austin_hr)*aus_prep['PRCP1']
    scooter_total_lr = (res_mean_lr[5]+res_mean_nr[8])*len(austin_hr)*aus_prep['PRCP2']
    scooter_total_nr = (res_mean_lr[5]+res_mean_nr[8])*len(austin_hr)*aus_prep['PRCP3']
    
    bike_total_hr = (res_mean_hr[6]+res_mean_hr[7])*len(austin_hr)*aus_prep['PRCP1']
    bike_total_lr = (res_mean_lr[6]+res_mean_nr[7])*len(austin_hr)*aus_prep['PRCP2']
    bike_total_nr = (res_mean_lr[6]+res_mean_nr[7])*len(austin_hr)*aus_prep['PRCP3']
    
    
    # modeshare weighted by rainy days
    avg_modeshare = (res_mean_hr*aus_prep['PRCP1'].values[0] + res_mean_lr*aus_prep['PRCP2'].values[0] +res_mean_nr*aus_prep['PRCP3'].values[0])/30
    scooter_ms = avg_modeshare[5]
    dlbike_ms = avg_modeshare[6]
    dkbike_ms = avg_modeshare[7]
    sc_transit_ms = avg_modeshare[8]
    print(avg_modeshare)

    # stop criteria
    sc = abs(aus_scooter_ms-scooter_ms)<= eps
    dl = abs(aus_dlbike_ms-dlbike_ms)<= eps
    dk = abs(aus_dkbike_ms-dkbike_ms)<= eps
    sctr = abs(aus_sc_transit_ms-sc_transit_ms)<= eps
    
    if sc and dl and dk and sctr:
        break

    beta_res['ASC_SCOOTER'] = beta_res['ASC_SCOOTER']+ math.log(aus_scooter_ms/scooter_ms)
    beta_res['ASC_DLBIKE'] = beta_res['ASC_DLBIKE']+ math.log(aus_dlbike_ms/dlbike_ms)
    beta_res['ASC_DKBIKE'] = beta_res['ASC_DKBIKE']+ math.log(aus_dkbike_ms/dkbike_ms)
    beta_res['ASC_SCTRANSIT'] = beta_res['ASC_SCTRANSIT']+ math.log(aus_sc_transit_ms/sc_transit_ms)
    
    
    i+=1
    print(i)

    

Prob. car                   0.709305
Prob. transit               0.042136
Prob. ridehailing           0.006723
Prob. walk                  0.119067
Prob. bike                  0.007471
Prob. scooter               0.034603
Prob. docklessbike          0.036516
Prob. dockedbike            0.010918
Prob. scooter+transit       0.033262
car time logit elas.       -0.281703
transit time logit elas.   -1.369909
taxi time logit elas.      -0.864472
car utility                -1.952107
dtype: float64
1
Prob. car                   0.789173
Prob. transit               0.048350
Prob. ridehailing           0.007874
Prob. walk                  0.136526
Prob. bike                  0.008405
Prob. scooter               0.005454
Prob. docklessbike          0.000104
Prob. dockedbike            0.000032
Prob. scooter+transit       0.004081
car time logit elas.       -0.213374
transit time logit elas.   -1.362207
taxi time logit elas.      -0.863570
car utility                -1.952107
dtype: float64
2
Prob

In [25]:
austin_hr.columns

Index(['TRACT', 'hhsize', 'child', 'hhincome', 'idincome', 'veh', 'bike',
       'TRPMILES', 'dest_tract', 'tour_id_unq', 'tour_id', 'MSA', 'TDTRPNUM',
       'TRACT.1', 'orig_tract', 'mean_idincome', 'mean_idincomek', 'higheredu',
       'employ', 'cartime', 'transittime', 'rdtime', 'walktime', 'biketime',
       'dltime', 'dbtime', 'sctime', 'sttime', 'stransittime', 'sccost',
       'dlcost', 'dbcost', 'stcost', 'sttotcost', 'sccost_adj', 'dlcost_adj',
       'dbcost_adj', 'stcost_adj', 'sttocost_adj', 'car_av', 'bike_av',
       'scooter_av', 'dlbike_av', 'dkbike_av', 'sctransit_av', 'transit_av',
       'rd_av', 'walk_av', 'PRCP_1', 'PRCP_2', 'PRCP_3', 'BKLN_1', 'BKLN_2',
       'BKLN_3', 'BKLN_4', 'SCAW', 'STAW', 'DLAW', 'DBAW', 'SCAV', 'STAV',
       'AVtech', 'DBDW', 'POP10_SQMI', 'popdensityk', 'BKLNless50',
       'trippurp_HBO', 'trippurp_HBSHOP', 'trippurp_HBSOC', 'trippurp_HBW',
       'trippurp_NHB'],
      dtype='object')

In [26]:
# add trip origin tract and home tract to calibration results
results_cali_hr['orig_tract'] = austin_hr['orig_tract']
results_cali_lr['orig_tract'] = austin_lr['orig_tract']
results_cali_nr['orig_tract'] = austin_nr['orig_tract']

results_cali_hr['tract'] = austin_hr['TRACT']
results_cali_lr['tract'] = austin_lr['TRACT']
results_cali_nr['tract'] = austin_nr['TRACT']


aus_pred_tract_hr = pd.DataFrame(results_cali_hr.groupby('orig_tract',as_index=False).mean())
aus_pred_tract_lr = pd.DataFrame(results_cali_lr.groupby('orig_tract',as_index=False).mean())
aus_pred_tract_nr = pd.DataFrame(results_cali_nr.groupby('orig_tract',as_index=False).mean())

aus_pred_tract_trip = pd.DataFrame(austin_hr.groupby('orig_tract',as_index=False)['orig_tract'].count())

# calculate trip count for each tract = prob * totoal number of triop
aus_pred_tract_hr['count_scooter']=aus_pred_tract_hr['Prob. scooter']*aus_pred_tract_trip['orig_tract']
aus_pred_tract_hr['count_dlbike']=aus_pred_tract_hr['Prob. docklessbike']*aus_pred_tract_trip['orig_tract']
aus_pred_tract_hr['count_dkbike']=aus_pred_tract_hr['Prob. dockedbike']*aus_pred_tract_trip['orig_tract']
aus_pred_tract_hr['count_sctransit']=aus_pred_tract_hr['Prob. scooter+transit']*aus_pred_tract_trip['orig_tract']

aus_pred_tract_lr['count_scooter']=aus_pred_tract_hr['Prob. scooter']*aus_pred_tract_trip['orig_tract']
aus_pred_tract_lr['count_dlbike']=aus_pred_tract_lr['Prob. docklessbike']*aus_pred_tract_trip['orig_tract']
aus_pred_tract_lr['count_dkbike']=aus_pred_tract_lr['Prob. dockedbike']*aus_pred_tract_trip['orig_tract']
aus_pred_tract_lr['count_sctransit']=aus_pred_tract_lr['Prob. scooter+transit']*aus_pred_tract_trip['orig_tract']

aus_pred_tract_nr['count_scooter']=aus_pred_tract_nr['Prob. scooter']*aus_pred_tract_trip['orig_tract']
aus_pred_tract_nr['count_dlbike']=aus_pred_tract_nr['Prob. docklessbike']*aus_pred_tract_trip['orig_tract']
aus_pred_tract_nr['count_dkbike']=aus_pred_tract_nr['Prob. dockedbike']*aus_pred_tract_trip['orig_tract']
aus_pred_tract_nr['count_sctransit']=aus_pred_tract_nr['Prob. scooter+transit']*aus_pred_tract_trip['orig_tract']

# number of rainy days
aus_pred_tract_hr[['count_scooter','count_dlbike','count_dkbike','count_sctransit']] = aus_pred_tract_hr[['count_scooter','count_dlbike','count_dkbike','count_sctransit']]*aus_prep['PRCP1'].values[0]
aus_pred_tract_lr[['count_scooter','count_dlbike','count_dkbike','count_sctransit']] = aus_pred_tract_lr[['count_scooter','count_dlbike','count_dkbike','count_sctransit']]*aus_prep['PRCP2'].values[0]
aus_pred_tract_nr[['count_scooter','count_dlbike','count_dkbike','count_sctransit']] = aus_pred_tract_nr[['count_scooter','count_dlbike','count_dkbike','count_sctransit']]*aus_prep['PRCP3'].values[0]

aus_pred_tract_30d = pd.DataFrame()
aus_pred_tract_30d['orig_tract'] = aus_pred_tract_hr['orig_tract']
aus_pred_tract_30d['count_scooter'] = (aus_pred_tract_hr['count_scooter'] + aus_pred_tract_lr['count_scooter'] + aus_pred_tract_nr['count_scooter'])/0.03
aus_pred_tract_30d['count_dlbike'] = (aus_pred_tract_hr['count_dlbike'] + aus_pred_tract_lr['count_dlbike'] + aus_pred_tract_nr['count_dlbike'])/0.03
aus_pred_tract_30d['count_dkbike'] = (aus_pred_tract_hr['count_dkbike'] + aus_pred_tract_lr['count_dkbike'] + aus_pred_tract_nr['count_dkbike'])/0.03
aus_pred_tract_30d['count_sctransit'] = (aus_pred_tract_hr['count_sctransit'] + aus_pred_tract_lr['count_sctransit'] + aus_pred_tract_nr['count_sctransit'])/0.03

aus_pred_tract_30d['scooter_tot'] = aus_pred_tract_30d['count_scooter'] + aus_pred_tract_30d['count_sctransit']
aus_pred_tract_30d['sharebike_tot'] = aus_pred_tract_30d['count_dlbike'] + aus_pred_tract_30d['count_dkbike']

aus_pred_tract_30d['scooter_tot'].sum()

aus_pred_tract_30d.sort_values('count_scooter', ascending = False).head(50)


,orig_tract,count_scooter,count_dlbike,count_dkbike,count_sctransit,scooter_tot,sharebike_tot
25,48453001100,10822.334051,219.635504,115.783844,10073.599915,20895.933966,335.419348
15,48453000603,10098.886883,163.707886,33.589361,9958.191516,20057.078398,197.297246
3,48453000204,8952.039269,0.000000,0.000000,5408.333351,14360.372620,0.000000
23,48453000902,8867.889538,177.652533,93.962282,6985.098393,15852.987931,271.614815
16,48453000604,8559.867656,140.960736,92.954047,8915.816639,17475.684295,233.914783
29,48453001305,8434.572814,175.645983,86.220816,8304.226398,16738.799212,261.866799
14,48453000601,8375.008715,151.263885,115.803552,8496.787418,16871.796133,267.067437
24,48453001000,8130.520583,119.868306,55.154409,5550.166352,13680.686935,175.022716
34,48453001403,6960.574115,152.485002,0.000000,5484.878560,12445.452675,152.485002
26,48453001200,6690.098273,129.328478,79.150938,7105.180983,13795.279255,208.479416


In [27]:
# check total number of mimo trips
aus_pred_tract_30d = aus_pred_tract_30d.merge(aus_total_ingen, left_on = 'orig_tract', right_on = 'GEOID10' )

aus_pred_tract_30d
print(aus_pred_tract_30d['ID_count'].sum())
print(aus_pred_tract_30d['scooter_tot'].sum())

435456
450456.880138384


In [28]:
aus_pred_tract_30d.to_csv('aus_pred_tract_30d.csv')

In [30]:
pickle.dump(beta_res, open('beta_res_mimo1227.pkl', 'wb'))